## Visualization of the estimation performances in AC magnetometry, with unknown frequency

In [ ]:
import numpy as np
import pandas as pd
import qsensoropt.plot_utils as pu
import matplotlib.pyplot as plt
from math import pi
from os.path import join
from os import makedirs

from qsensoropt.utils import normalize

The following parameters are the one passed to the function
`complete_Bfield_estimation` of the `nv_center_ac_omega` file.

In [ ]:
#####################
batch_size = 192
max_resources = 24
learning_rate = 1e-4
res = "meas"
log_loss = True
cumulative_loss = True
invT2 = 0.0
#####################

In [ ]:
def generate_sim_name(
    batch_size, max_resources,
    learning_rate: float = 1e-2,
    cumulative_loss: bool = False,
    log_loss: bool = False,
    res: str = "meas", invT2: float = None,
    two_parameters: bool = False,
    pgh=False, sigma=False,
    sigmah=False, static=False,
) -> str:
    """Composes the controls in the
    above cells and produces
    the name of the simulation.
    """
    name = "nv_center_ac_omega_" + res
    name += f"_invT2_{invT2:.2f}"
    name += f"_lr_{learning_rate}"
    if pgh:
        name += '_PGH'
    elif sigma:
        name += '_Sigma'
    elif sigmah:
        name += '_SigmaH'
    elif static:
        name += '_static'
        
    name += f'_batchsize_{batch_size}_num_steps_'+\
        f'{max_resources}_max_resources_{max_resources:.2f}'
    name += f'_ll_{log_loss}_cl_{cumulative_loss}'
    return name

In [ ]:
name = generate_sim_name(
    batch_size, max_resources=max_resources,
    learning_rate=learning_rate,
    cumulative_loss=cumulative_loss,
    log_loss=log_loss,
    res=res, invT2=invT2,
)
name_pgh = generate_sim_name(
    batch_size, max_resources=max_resources,
    learning_rate=learning_rate,
    cumulative_loss=cumulative_loss,
    log_loss=log_loss,
    res=res, invT2=invT2,
    pgh=True,
)
name_sigma = generate_sim_name(
    batch_size, max_resources=max_resources,
    learning_rate=learning_rate,
    cumulative_loss=cumulative_loss,
    log_loss=log_loss,
    res=res, invT2=invT2,
    sigma=True,
)
name_sigmah = generate_sim_name(
    batch_size, max_resources=max_resources,
    learning_rate=learning_rate,
    cumulative_loss=cumulative_loss,
    log_loss=log_loss,
    res=res, invT2=invT2,
    sigmah=True,
)
name_static = generate_sim_name(
    batch_size, max_resources=max_resources,
    learning_rate=learning_rate,
    cumulative_loss=cumulative_loss,
    log_loss=log_loss,
    res=res, invT2=invT2,
    static=True,
)

### Precision vs. Resources

In [ ]:
prec_df = pd.read_csv(
    f'./nv_center_ac_omega/data/{name}_eval.csv',
)
prec_pgh_df = pd.read_csv(
    f'./nv_center_ac_omega/data/{name_pgh}_eval.csv',
)
prec_sigma_df = pd.read_csv(
    f'./nv_center_ac_omega/data/{name_sigma}_eval.csv',
)
prec_sigmah_df = pd.read_csv(
    f'./nv_center_ac_omega/data/{name_sigmah}_eval.csv',
)
prec_static_df = pd.read_csv(
    f'./nv_center_ac_omega/data/{name_static}_eval.csv',
)
if res=="meas":
    prec_df["MeasStep"] = prec_df["Resources"]
    prec_pgh_df["MeasStep"] = prec_pgh_df["Resources"]
    prec_sigma_df["MeasStep"] = prec_sigma_df["Resources"]
    prec_sigmah_df["MeasStep"] = prec_sigmah_df["Resources"]
    prec_static_df["MeasStep"] = prec_static_df["Resources"]
    xlabel = "MeasStep"
else:
    prec_df["Time"] = prec_df["Resources"]
    prec_pgh_df["Time"] = prec_pgh_df["Resources"]
    prec_sigma_df["Time"] = prec_sigma_df["Resources"]
    prec_sigmah_df["Time"] = prec_sigmah_df["Resources"]
    prec_static_df["Time"] = prec_static_df["Resources"]
    xlabel = "Time"

In [ ]:
img_dir = f'./nv_center_ac_omega/images/{name}'
makedirs(img_dir, exist_ok=True)

In [ ]:
pu.plot_multiples(
        [prec_df, prec_static_df, prec_sigmah_df,
         prec_sigma_df, prec_pgh_df], 
        xlabel, "MSE", 
        ["Neural Network", "Static", r"$\sigma^{-1}$",
         r"$\sigma^{-1}$ & $T^{-1}$", "PGH"],
        figsize=(3, 2), dpi=200,
        #title="Precision",
        path=img_dir,
    )

### Trajectories

In [ ]:
##################
traj_num = 5
static = False
##################

In [ ]:
if static:
    traj_num = 1

In [ ]:
name_traj = name_static if static else name
data_df = pd.read_csv(
    f'./nv_center_ac_omega/data/{name_traj}_ext.csv',
)
data_df['ResOverMaxRes'] = (data_df['ResOverMaxRes']+1)/2
data_df['Resources'] = max_resources*data_df['ResOverMaxRes']
if res=="meas":
    data_df["MeasStep"] = data_df['Resources']
    xlabel = "MeasStep"
else:
    data_df["Time"] = data_df['Resources']
    xlabel = "Time"
data_df["Tau"] = data_df['EvolutionTime']
data_df['Bcolor'] = (normalize(data_df['B'], (1.0, 2.0))+1)/2

In [ ]:
trajectory_list = [
    data_df[data_df['Estimation']==i]
    for i in range(traj_num)
]

In [ ]:
omega_list = [
    f"{trajectory_list[i]['omega'].values[0]:.2f}"
    for i in range(traj_num)
]

In [ ]:
pu.plot_multiples(
    trajectory_list, xlabel, "Tau",
    omega_list, log_scale=False,
    title="Control strategy",
    figsize=(3, 2.2), dpi=200,
    legend_location="lower right",
    path=img_dir,
)

### Small plots

In [ ]:
pu.plot_3_values(
    data_df, 
    "Mean_omega", "Tau", color="ResOverMaxRes",
    figsize=(2, 2), dpi=200, 
    path=img_dir,
)

In [ ]:
pu.plot_3_values(
    data_df, 
    "LogDev_omega", "Tau", color="ResOverMaxRes", 
    figsize=(2, 2), dpi=200,
    path=img_dir, 
)

In [ ]:
pu.plot_3_values(
    data_df, 
    "LogDev_omega", "Tau", color="Bcolor", 
    figsize=(2, 2), dpi=200,
    path=img_dir,
)

In [ ]:
pu.plot_3_values(
    data_df, 
    "Mean_B", "Tau", color="ResOverMaxRes", 
    figsize=(2, 2), dpi=200,
    path=img_dir,
)
pu.plot_3_values(
    data_df, 
    "Corr_omega_B", "Tau", color="ResOverMaxRes",
    figsize=(2, 2), dpi=200, 
    path=img_dir,
)
pu.plot_3_values(
    data_df, 
    "LogDev_B", "Tau", color="ResOverMaxRes", 
    figsize=(2, 2), dpi=200,
    path=img_dir
)